# 클러스터 비교 통계 분석

In [1]:
import sys
from pathlib import Path

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from src.loading import DataLoader
from src.utils.statistic import Chi2Test

loader2 = DataLoader(
    output_file=DATA_DIR / 'maude_clustered.parquet',
    max_workers=4
)

In [2]:
import polars as pl
import pandas as pd

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}

df = loader2.load(adapter=adapter, **polars_kwargs)


📖 /Users/t2023-m0103/Desktop/sparta/final_project/data/maude_clustered.parquet 로딩 중... (adapter=polars)


In [3]:
df.head().collect()

mdr_report_key,adverse_event_flag,product_problem_flag,date_occurred,date_received,date_manufactured,event_type,previous_use_flag,single_use_flag,reprocessed_and_reused_flag,product_problems,manufacturer_name,brand_name,model_number,udi_di,product_code,operator,product_name,patient_age,mdr_text,outcome_L,outcome_H,outcome_S,outcome_C,outcome_R,outcome_D,outcome_O,patient_harm,problem_components,defect_confirmed,defect_type,mdr_sntc,embeddings,cluster
i32,bool,bool,date,date,date,cat,bool,bool,bool,str,str,str,str,str,str,bool,str,i64,str,bool,bool,bool,bool,bool,bool,bool,str,list[str],bool,str,str,list[f64],i32
16081309,true,false,2022-06-27,2023-01-01,2022-02-15,"""Injury""",true,true,false,"""['Adverse Event Without Identi…","""ST JUDE MEDICAL""","""PROCLAIM""","""DRG""","""05415067020215""","""PMP""",true,"""dorsal root ganglion stimulato…",null,"""[Description of Event or Probl…",false,false,false,false,false,false,true,"""Minor Injury""","[""ipg implant site"", ""surgical intervention pending"", … ""implant site complication""]",false,"""Other""","""Minor Injury. ipg implant site…","[-0.018616, -0.040333, … 0.024962]",2
16081313,true,false,2022-11-01,2023-01-01,2019-10-30,"""Injury""",true,true,false,"""['Adverse Event Without Identi…","""ST JUDE MEDICAL""","""ASSURITY MRI""","""PM2272""","""05414734509589""","""LWP""",true,"""implantable pulse generator pa…",null,"""[Description of Event or Probl…",false,false,false,false,false,false,true,"""No Harm""","[""laser adhesion preparation field safety correction action"", ""device explantation"", … ""device subject to safety correction action""]",true,"""Other""","""No Harm. laser adhesion prepar…","[-0.023548, -0.028649, … 0.014175]",-1
16081342,true,false,2022-11-01,2023-01-01,2021-12-20,"""Injury""",true,true,false,"""['Device Appears to Trigger Re…","""ALLERGAN""","""INSPIRA SMOOTH SILICONE GEL FI…","""SSF-415""","""10888628033160""","""FTR""",true,"""prosthesis breast noninflatabl…",11680,"""[Additional Manufacturer Narra…",false,false,false,false,true,false,false,"""Serious Injury""","[""Device Appears to Trigger Rejection"", ""Capsular Contracture Baker Grade IV"", … ""Device Appears to Trigger Rejection""]",false,"""Other""","""Serious Injury. Device Appears…","[-0.008962, -0.032672, … 0.02119]",-1
16081344,true,false,2022-11-01,2023-01-01,2021-12-20,"""Injury""",true,true,false,"""['Device Appears to Trigger Re…","""ALLERGAN""","""INSPIRA SMOOTH SILICONE GEL FI…","""SSF-415""","""10888628033160""","""FTR""",true,"""prosthesis breast noninflatabl…",11680,"""[Additional Manufacturer Narra…",false,false,false,false,true,false,false,"""Serious Injury""","[""Device Appears to Trigger Rejection"", ""Capsular Contracture Baker Grade IV"", … ""Device Appears to Trigger Rejection""]",false,"""Other""","""Serious Injury. Device Appears…","[-0.008962, -0.032672, … 0.02119]",-1
16081345,true,true,2022-09-01,2023-01-01,2021-08-28,"""Injury""",true,true,false,"""['Material Rupture', 'Migratio…","""MENTOR TEXAS LP""","""MENTOR MEMORYGEL XTRA BREAST I…","""SHPX380""","""00081317025269""","""FTR""",true,"""prosthesis breast noninflatabl…",10220,"""[Additional Manufacturer Narra…",false,false,false,false,true,false,false,"""Serious Injury""","[""Breast Prosthesis"", ""Material Rupture"", … ""Device Structure""]",false,"""Mechanical/Structural""","""Serious Injury. Breast Prosthe…","[-0.012143, -0.034709, … 0.019714]",2


## 카이제곱검정

In [4]:
chi2test = Chi2Test()
chi2_result = chi2test.execute(df.collect().to_pandas(), 'cluster', 'patient_harm', alpha=0.05)


[기대빈도 확인]
----------------------------------------
최소 기대빈도: 275.60
5 미만 셀: 0/25 (0.0%)
✅ 카이제곱검정 사용 가능

[기대빈도]
patient_harm    Death  Minor Injury    No Harm  Serious Injury   Unknown
cluster                                                                 
-1            2296.50       6947.51  121555.18        20150.82  49957.99
 0             406.43       1229.55   21512.41         3566.22   8841.39
 1            2162.58       6542.36  114466.66        18975.72  47044.68
 2             678.89       2053.81   35933.89         5956.94  14768.48
 3             275.60        833.77   14587.86         2418.30   5995.47

[표준화 잔차]


patient_harm,Death,Minor Injury,No Harm,Serious Injury,Unknown
cluster,,,,,
-1,-11.24,-73.31,-30.52,-86.07,132.02
0,-19.91,-35.06,-145.44,-59.70,282.12
1,39.64,-78.89,203.84,-129.56,-214.74
2,-24.56,321.15,-185.14,465.49,-121.34
3,-15.88,-28.84,-15.69,-46.70,68.29


(|잔차| > 2: 유의한 차이, |잔차| > 3: 매우 강한 연관성)

[결론]
✅ p-value(0.0000) < 0.05 → 귀무가설 기각
   cluster과(와) patient_harm은(는) 관련이 있음   효과 크기: 강한 관계

[사후분석]
   표준화 잔차 |값| > 2인 셀 해석:
   표준화 잔차 |값| > 2인 셀 해석:
   • -1 - Death: 예상보다 적음 (잔차=-11.24)
   • -1 - Minor Injury: 예상보다 적음 (잔차=-73.31)
   • -1 - No Harm: 예상보다 적음 (잔차=-30.52)
   • -1 - Serious Injury: 예상보다 적음 (잔차=-86.07)
   • -1 - Unknown: 예상보다 많음 (잔차=132.02)
   • 0 - Death: 예상보다 적음 (잔차=-19.91)
   • 0 - Minor Injury: 예상보다 적음 (잔차=-35.06)
   • 0 - No Harm: 예상보다 적음 (잔차=-145.44)
   • 0 - Serious Injury: 예상보다 적음 (잔차=-59.70)
   • 0 - Unknown: 예상보다 많음 (잔차=282.12)
   • 1 - Death: 예상보다 많음 (잔차=39.64)
   • 1 - Minor Injury: 예상보다 적음 (잔차=-78.89)
   • 1 - No Harm: 예상보다 많음 (잔차=203.84)
   • 1 - Serious Injury: 예상보다 적음 (잔차=-129.56)
   • 1 - Unknown: 예상보다 적음 (잔차=-214.74)
   • 2 - Death: 예상보다 적음 (잔차=-24.56)
   • 2 - Minor Injury: 예상보다 많음 (잔차=321.15)
   • 2 - No Harm: 예상보다 적음 (잔차=-185.14)
   • 2 - Serious Injury: 예상보다 많음 (잔차=465.49)
   • 2 - Unknown: 예상보다 적음 (잔차=-121.34

In [5]:
# 전체 제품 코드에서 이상 사례 발생 수가 높은 상위 10개 제품 코드 확인
top10_result = (
    df
    .group_by("product_code")
    .agg(pl.count().alias("total_count"))
    .sort("total_count", descending=True)
    .head(10)
    .collect()
)

# 상위 10개 제품 코드 리스트화
top10_product_codes = top10_result['product_code'].to_list()

chi2_result = chi2test.execute(
    df
    .filter(pl.col("product_code").is_in(top10_product_codes))
    .collect().to_pandas(), 
    'cluster', 
    'product_code', 
    alpha=0.05)

/var/folders/jz/7b91vg4d00z75sw7vt8q3c3h0000gn/T/ipykernel_54289/3432440736.py:5: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("total_count"))



[기대빈도 확인]
----------------------------------------
최소 기대빈도: 742.25
5 미만 셀: 0/50 (0.0%)
✅ 카이제곱검정 사용 가능

[기대빈도]
product_code      DSQ       DTB       FTR       LGW      LWP       LWS  \
cluster                                                                  
-1            7659.02  21284.82  12230.93  28982.32  8608.04  16645.16   
 0            1173.82   3262.11   1874.51   4441.83  1319.27   2551.04   
 1            6551.36  18206.56  10462.07  24790.83  7363.12  14237.90   
 2            2102.14   5841.95   3356.97   7954.65  2362.61   4568.52   
 3             825.66   2294.56   1318.52   3124.37   927.97   1794.39   

product_code      NIK       NVN      NVY      OZD  
cluster                                            
-1            9030.05  16264.55  6885.26  9020.85  
 0            1383.95   2492.70  1055.23  1382.54  
 1            7724.10  13912.33  5889.50  7716.23  
 2            2478.44   4464.06  1889.77  2475.91  
 3             973.46   1753.36   742.25   972.47  

[표준화 

product_code,DSQ,DTB,FTR,LGW,LWP,LWS,NIK,NVN,NVY,OZD
cluster,,,,,,,,,,
-1,-4.08,40.89,-25.44,30.71,1.22,-6.15,-8.73,-26.37,-15.42,-19.68
0,11.71,42.39,-1.56,-15.20,-26.00,17.50,-4.00,-16.88,-3.42,-15.56
1,12.25,-38.09,-45.75,-48.29,25.73,19.44,24.50,48.94,25.85,22.70
2,-13.44,-42.55,122.18,19.41,-28.98,-36.09,-20.22,-16.55,-5.75,25.48
3,-14.60,0.11,13.26,29.66,1.05,0.68,-5.37,-10.99,-12.60,-26.12


(|잔차| > 2: 유의한 차이, |잔차| > 3: 매우 강한 연관성)

[결론]
✅ p-value(0.0000) < 0.05 → 귀무가설 기각
   cluster과(와) product_code은(는) 관련이 있음   효과 크기: 약한 관계

[사후분석]
   표준화 잔차 |값| > 2인 셀 해석:
   표준화 잔차 |값| > 2인 셀 해석:
   • -1 - DSQ: 예상보다 적음 (잔차=-4.08)
   • -1 - DTB: 예상보다 많음 (잔차=40.89)
   • -1 - FTR: 예상보다 적음 (잔차=-25.44)
   • -1 - LGW: 예상보다 많음 (잔차=30.71)
   • -1 - LWS: 예상보다 적음 (잔차=-6.15)
   • -1 - NIK: 예상보다 적음 (잔차=-8.73)
   • -1 - NVN: 예상보다 적음 (잔차=-26.37)
   • -1 - NVY: 예상보다 적음 (잔차=-15.42)
   • -1 - OZD: 예상보다 적음 (잔차=-19.68)
   • 0 - DSQ: 예상보다 많음 (잔차=11.71)
   • 0 - DTB: 예상보다 많음 (잔차=42.39)
   • 0 - LGW: 예상보다 적음 (잔차=-15.20)
   • 0 - LWP: 예상보다 적음 (잔차=-26.00)
   • 0 - LWS: 예상보다 많음 (잔차=17.50)
   • 0 - NIK: 예상보다 적음 (잔차=-4.00)
   • 0 - NVN: 예상보다 적음 (잔차=-16.88)
   • 0 - NVY: 예상보다 적음 (잔차=-3.42)
   • 0 - OZD: 예상보다 적음 (잔차=-15.56)
   • 1 - DSQ: 예상보다 많음 (잔차=12.25)
   • 1 - DTB: 예상보다 적음 (잔차=-38.09)
   • 1 - FTR: 예상보다 적음 (잔차=-45.75)
   • 1 - LGW: 예상보다 적음 (잔차=-48.29)
   • 1 - LWP: 예상보다 많음 (잔차=25.73)
   • 1 - LWS: 예상보다 많음 (잔차=19

In [6]:
chi2_result = chi2test.execute(df.collect().to_pandas(), 'cluster', 'defect_type', alpha=0.05)


[기대빈도 확인]
----------------------------------------
최소 기대빈도: 35.94
5 미만 셀: 0/65 (0.0%)
✅ 카이제곱검정 사용 가능

[기대빈도]
defect_type  Alarm/Alert  Communication/Connectivity  Electrical/Power  \
cluster                                                                  
-1               1029.87                     4862.51          16329.63   
 0                182.26                      860.55           2889.96   
 1                969.82                     4578.95          15377.36   
 2                304.45                     1437.44           4827.33   
 3                123.60                      583.55           1959.72   

defect_type  Environmental/Compatibility  Functional Failure  \
cluster                                                        
-1                               1921.25            18242.19   
 0                                340.02             3228.44   
 1                               1809.21            17178.40   
 2                                567.96           

defect_type,Alarm/Alert,Communication/Connectivity,Electrical/Power,Environmental/Compatibility,Functional Failure,Labeling/Packaging,Mechanical/Structural,Other,Sensor/Accuracy,Software/Interface,Sterility/Contamination,Unknown,User/Human Factor
cluster,,,,,,,,,,,,,
-1,-9.81,19.60,3.10,-11.50,-44.03,-6.79,-63.80,21.89,10.98,-27.13,-31.75,81.58,-20.12
0,-11.43,-24.09,-46.21,4.17,-38.59,-5.08,-41.43,-16.07,-29.35,-17.20,-4.50,197.98,-14.36
1,18.95,9.00,36.92,17.23,56.08,15.90,47.25,-68.93,51.36,51.33,1.05,-137.28,22.88
2,0.15,-18.53,-52.52,-4.11,19.83,-8.24,76.41,63.10,-57.32,-18.48,67.66,-80.60,17.33
3,-11.12,-23.45,26.16,-13.67,-14.28,-5.83,-17.82,50.39,-49.96,-15.60,-12.03,35.14,-15.76


(|잔차| > 2: 유의한 차이, |잔차| > 3: 매우 강한 연관성)

[결론]
✅ p-value(0.0000) < 0.05 → 귀무가설 기각
   cluster과(와) defect_type은(는) 관련이 있음   효과 크기: 약한 관계

[사후분석]
   표준화 잔차 |값| > 2인 셀 해석:
   표준화 잔차 |값| > 2인 셀 해석:
   • -1 - Alarm/Alert: 예상보다 적음 (잔차=-9.81)
   • -1 - Communication/Connectivity: 예상보다 많음 (잔차=19.60)
   • -1 - Electrical/Power: 예상보다 많음 (잔차=3.10)
   • -1 - Environmental/Compatibility: 예상보다 적음 (잔차=-11.50)
   • -1 - Functional Failure: 예상보다 적음 (잔차=-44.03)
   • -1 - Labeling/Packaging: 예상보다 적음 (잔차=-6.79)
   • -1 - Mechanical/Structural: 예상보다 적음 (잔차=-63.80)
   • -1 - Other: 예상보다 많음 (잔차=21.89)
   • -1 - Sensor/Accuracy: 예상보다 많음 (잔차=10.98)
   • -1 - Software/Interface: 예상보다 적음 (잔차=-27.13)
   • -1 - Sterility/Contamination: 예상보다 적음 (잔차=-31.75)
   • -1 - Unknown: 예상보다 많음 (잔차=81.58)
   • -1 - User/Human Factor: 예상보다 적음 (잔차=-20.12)
   • 0 - Alarm/Alert: 예상보다 적음 (잔차=-11.43)
   • 0 - Communication/Connectivity: 예상보다 적음 (잔차=-24.09)
   • 0 - Electrical/Power: 예상보다 적음 (잔차=-46.21)
   • 0 - Environmental/Compatibi

In [7]:
# 전체 제품 코드에서 이상 사례 발생 수가 높은 상위 10개 제품 코드 확인
top10_result = (
    df
    .group_by("manufacturer_name")
    .agg(pl.count().alias("total_count"))
    .sort("total_count", descending=True)
    .head(10)
    .collect()
)

# 상위 10개 제품 코드 리스트화
top10_manufacturer_names = top10_result['manufacturer_name'].to_list()

chi2_result = chi2test.execute(
    df
    .filter(pl.col("manufacturer_name").is_in(top10_manufacturer_names))
    .collect().to_pandas(), 
    'cluster', 
    'manufacturer_name', 
    alpha=0.05)

/var/folders/jz/7b91vg4d00z75sw7vt8q3c3h0000gn/T/ipykernel_54289/463094400.py:5: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("total_count"))



[기대빈도 확인]
----------------------------------------
최소 기대빈도: 406.89
5 미만 셀: 0/50 (0.0%)
✅ 카이제곱검정 사용 가능

[기대빈도]
manufacturer_name  ABBOTT VASCULAR  ABIOMED  BIOTRONIK SE &  KG  \
cluster                                                           
-1                         5655.46  9107.15             4027.43   
 0                          975.66  1571.13              694.80   
 1                         5292.90  8523.31             3769.24   
 2                         1601.61  2579.12             1140.56   
 3                          668.37  1076.29              475.97   

manufacturer_name  BOSTON SCIENTIFIC  BOSTON SCIENTIFIC NEUROMODULATION  \
cluster                                                                   
-1                          27098.71                            7797.47   
 0                           4674.98                            1345.19   
 1                          25361.47                            7297.59   
 2                           7674.28        

manufacturer_name,ABBOTT VASCULAR,ABIOMED,BIOTRONIK SE & KG,BOSTON SCIENTIFIC,BOSTON SCIENTIFIC NEUROMODULATION,EDWARDS LIFESCIENCES,MEDTRONIC,MENTOR TEXAS LP,ST JUDE MEDICAL,THORATEC
cluster,,,,,,,,,,
-1,2.44,-18.00,23.22,-27.85,42.28,-18.71,18.75,-24.78,-0.18,2.35
0,-15.71,-17.92,-0.18,-50.75,-22.61,23.80,85.29,-3.20,-44.18,1.43
1,3.42,21.60,-28.26,80.33,-43.34,2.62,-22.26,-47.46,-2.35,9.76
2,-19.46,26.62,-12.96,-42.05,26.26,20.98,-62.11,123.56,30.58,-12.00
3,32.36,-27.96,32.27,-18.63,-14.35,-14.17,1.20,18.21,13.15,-17.48


(|잔차| > 2: 유의한 차이, |잔차| > 3: 매우 강한 연관성)

[결론]
✅ p-value(0.0000) < 0.05 → 귀무가설 기각
   cluster과(와) manufacturer_name은(는) 관련이 있음   효과 크기: 약한 관계

[사후분석]
   표준화 잔차 |값| > 2인 셀 해석:
   표준화 잔차 |값| > 2인 셀 해석:
   • -1 - ABBOTT VASCULAR: 예상보다 많음 (잔차=2.44)
   • -1 - ABIOMED: 예상보다 적음 (잔차=-18.00)
   • -1 - BIOTRONIK SE &  KG: 예상보다 많음 (잔차=23.22)
   • -1 - BOSTON SCIENTIFIC: 예상보다 적음 (잔차=-27.85)
   • -1 - BOSTON SCIENTIFIC NEUROMODULATION: 예상보다 많음 (잔차=42.28)
   • -1 - EDWARDS LIFESCIENCES: 예상보다 적음 (잔차=-18.71)
   • -1 - MEDTRONIC: 예상보다 많음 (잔차=18.75)
   • -1 - MENTOR TEXAS LP: 예상보다 적음 (잔차=-24.78)
   • -1 - THORATEC: 예상보다 많음 (잔차=2.35)
   • 0 - ABBOTT VASCULAR: 예상보다 적음 (잔차=-15.71)
   • 0 - ABIOMED: 예상보다 적음 (잔차=-17.92)
   • 0 - BOSTON SCIENTIFIC: 예상보다 적음 (잔차=-50.75)
   • 0 - BOSTON SCIENTIFIC NEUROMODULATION: 예상보다 적음 (잔차=-22.61)
   • 0 - EDWARDS LIFESCIENCES: 예상보다 많음 (잔차=23.80)
   • 0 - MEDTRONIC: 예상보다 많음 (잔차=85.29)
   • 0 - MENTOR TEXAS LP: 예상보다 적음 (잔차=-3.20)
   • 0 - ST JUDE MEDICAL: 예상보다 적음 (잔차=-44.18)
   